In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics import classification_report

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Josvaldes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Josvaldes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Leer el archivo CSV
df = pd.read_csv('C:/Users/Josvaldes/Documents/Maestria/Austral/2ano/textMining/proyecto/TextMining/TextMining/RECLAMOS 3.csv', encoding='utf-8')
df.head()

,problema_id,descripcion,alias_8_nombre3,problema_n,obsitem,Concaobsitem
0,70e03aaf-6cfb-42e8-92ae-fcb46ff479dd,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,B1 - Pérdidas de gas con piezas dañadas,HAY UNA PERILLA DE LAS HORNALLA QUE NO SE PUED...,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
1,e5be47a0-c506-4a79-87e9-2a45f06d496c,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,G - Perilla no gira / gira mal,HAY UNA PERILLA DE LAS HORNALLA QUE NO SE PUED...,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
2,2b2af341-20c5-4607-baa0-01972617647e,TERMO ELECTRICO EL-55,ELE - Termo eléctrico,D - No enciende,DESPUÉS DE QUE UNA PERSONA SE DUCHO NO VOLVIÓ ...,"TERMO ELECTRICO EL-55 ELE - Termo eléctrico "" ..."
3,ab0ed03a-8aab-4317-9aad-fcc06dd41520,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,D2 - Hornalla Mal funcionamiento,NaN,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...
4,ab0ed03a-8aab-4317-9aad-fcc06dd41520,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.),GN - Gas Natural,D2 - Hornalla Mal funcionamiento,LAS HORNALLAS TARDAN MUCHÍSIMO TIEMPO EN PREND...,COCINA PALACE CRISTAL BLACK LX S2 (GN) (U.) GN...


In [4]:
# Buscar y reemplazar la categoría
categoria_a_buscar = 'B2 - Pérdidas de gas sin piezas dañadas'
categoria_a_reemplazar = 'B1 - Pérdidas de gas con piezas dañadas'

df['problema_n'] = df['problema_n'].replace(categoria_a_buscar, categoria_a_reemplazar)

In [5]:
# Se elimina las filas vacias de los campos descripcion, alias_8_nombre3  y obsitem
df.dropna(subset=['descripcion', 'alias_8_nombre3','obsitem'], inplace=True)

In [6]:
# Concatenar las columnas "descripcion", "alias_8_nombre3" y "obsitem" en una nueva columna "Concaobsitem"
df['Concaobsitem'] = df['descripcion'] + ' ' + df['alias_8_nombre3']+ ' ' + df['obsitem']

In [7]:
# Obtener las columnas de diagnósticos y códigos AIS
descripciones = df['Concaobsitem'].tolist()
etiquetas = df['problema_n'].tolist()
type(descripciones)

list

In [8]:
for desc in descripciones:
    if not isinstance(desc, str):
        print(f"Elemento {desc} no es una cadena de texto (str).")

In [9]:
# Codificar las etiquetas como etiquetas numéricas
label_encoder = LabelEncoder()
etiquetas_numericas = label_encoder.fit_transform(etiquetas)

In [10]:
# Preprocesamiento de texto
stemmer = SnowballStemmer('spanish')
stop_words = set(stopwords.words('spanish'))

In [11]:
def preprocesar_texto(texto):
    palabras = nltk.word_tokenize(texto.lower())
    palabras = [stemmer.stem(palabra) for palabra in palabras if palabra.isalpha() and palabra not in stop_words]
    return ' '.join(palabras)

In [12]:
descripciones_preprocesadas = [preprocesar_texto(desc) for desc in descripciones]

In [13]:
# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(descripciones_preprocesadas, etiquetas_numericas, test_size=0.2, random_state=42)

In [14]:
# Creación del pipeline para el clasificador basado en texto
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),  # Experimenta con diferentes valores para max_features
    ('clf', SVC(kernel='linear'))  # Experimenta con diferentes kernels
])

In [15]:
# Entrenamiento del modelo
pipeline.fit(X_train, y_train)
scores = cross_val_score(pipeline, X_train, y_train, cv=5) 

c:\Users\Josvaldes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [16]:
print("Accuracy en validación cruzada:", scores.mean())

Accuracy en validación cruzada: 0.6612213740458015


In [17]:
# Evaluación del modelo en el conjunto de prueba
y_pred = pipeline.predict(X_test)
report = classification_report(y_test, y_pred, labels=np.unique(etiquetas_numericas), target_names=label_encoder.inverse_transform(np.unique(etiquetas_numericas)), zero_division=1)
print(report)

                                                                                                   precision    recall  f1-score   support

                                                                    B - Descargas de electricidad       0.83      0.22      0.34        23
                                               B - La llama no enciende al abrir el grifo de agua       0.65      0.95      0.77        37
                                                      B - Pérdidas de gas sin/con piezas quemadas       1.00      0.00      0.00         7
                                                    B - Pérdidas de gas – sin/con piezas quemadas       0.25      0.25      0.25         4
                                                          B1 - Pérdidas de gas con piezas dañadas       0.76      0.79      0.77       277
                              C - La llama se apaga luego de algunos minutos de estar funcionando       0.50      0.54      0.52        13
                          